In [ ]:
import requests
import pandas as pd
import json

# Fetch data from APIs
def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# API Endpoints
quiz_data_url = "https://www.jsonkeeper.com/b/LLQT"
quiz_submission_url = "https://api.jsonserve.com/rJvd7g"
history_data_url = "https://api.jsonserve.com/XgAgFJ"

# Load Data
quiz_data = fetch_data(quiz_data_url)
quiz_submission_data = fetch_data(quiz_submission_url)
history_data = fetch_data(history_data_url)

# Convert historical data to DataFrame
history_df = pd.DataFrame(history_data)

# Analyze Weak Areas
def analyze_weakness(history_df):
    topic_errors = {}
    for quiz in history_df['quizzes']:
        for question in quiz['questions']:
            topic = question['topic']
            if question['selected_option'] != question['correct_option']:
                topic_errors[topic] = topic_errors.get(topic, 0) + 1
    weak_topics = sorted(topic_errors.items(), key=lambda x: x[1], reverse=True)
    return weak_topics[:3]  # Top 3 weak topics

# Analyze Strengths
def analyze_strengths(history_df):
    topic_correct = {}
    for quiz in history_df['quizzes']:
        for question in quiz['questions']:
            topic = question['topic']
            if question['selected_option'] == question['correct_option']:
                topic_correct[topic] = topic_correct.get(topic, 0) + 1
    strong_topics = sorted(topic_correct.items(), key=lambda x: x[1], reverse=True)
    return strong_topics[:3]  # Top 3 strong topics

# Generate Recommendations
def generate_recommendations(weak_topics):
    recommendations = []
    for topic, _ in weak_topics:
        recommendations.append(f"Focus on revising {topic}. Attempt more practice questions.")
    return recommendations

# Student Persona Analysis
def student_persona(history_df):
    avg_score = sum([quiz['score'] for quiz in history_df['quizzes']]) / len(history_df['quizzes'])
    if avg_score > 80:
        return "High Achiever: Keep pushing your limits!"
    elif avg_score > 50:
        return "Intermediate: You are doing well, but improvement is needed."
    else:
        return "Beginner: Focus on foundational topics first."

# Run Analysis
weak_topics = analyze_weakness(history_df)
strong_topics = analyze_strengths(history_df)
recommendations = generate_recommendations(weak_topics)
persona = student_persona(history_df)

# Display Results
print("Weak Topics:", weak_topics)
print("Strong Topics:", strong_topics)
print("Recommendations:")
for rec in recommendations:
    print("-", rec)
print("Student Persona:", persona)
